# Zipline 滑價調整

在真實世界交易時，因為股價隨時都在變動，下單時的些微時間差也可能造成預期的價格與成交價有落差，而這個價差就是滑價。

TQuant Lab 提供使用者四種不同 slippage model 模擬這項因素：

### zipline.finance.slippage.FixedSlippage:

自動在每筆交易加入 $\pm \frac{spread}{2}$。如果是買入 $+ \frac{spread}{2}$，成交價 ，賣出 $- \frac{spread}{2}$，也就是模擬較不利的情況。
    
#### Parameters:
* spread: _float_, optional
        決定上述 spread 大小。
                
### zipline.finance.slippage.VolumeShareSlippage
    
根據當日股票的流動性決定滑價大小，滑價後成交價計算方法如下:
    
$$
price \times [1 \pm {price\_impact}) \times ({volume\_share}^2)], \\
price=當日收盤價, \\
volume\_share=此單交易量佔總交易量百分比數，最高為 {volume\_limit}。
$$ 
    
#### Parameters:
* volume_limit: _float_, optional
        買賣量最高佔總交易量的百分比，預設 = 2.5%。
* price_impact: _float_, optional
        滑價影響程度，其值越大時，滑價程度越大，預設 = 0.1。
            
### zipline.finance.slippage.FixedBasisPointsSlippage

設定固定基點的滑價，其計算方法為:
    
$$
price \times [(1 \pm basis\_points \times 0.0001)]
$$

設定當日交易量限制:
    
$$
historical\_volume \times volume\_limit, \\
historical\_volume=當日成交量\\
$$

#### Parameters:
* basis_point: _float_, optional
        設置滑價基點，基點越大，滑價程度越大，預設 = 5.0。
* volume_limit: _float_, optional    
        買賣量最高佔總交易量的百分比，預設 = 10%。
            
### zipline.finance.slippage.NoSlippage

不設置滑價。

## 編程方法
滑價於 initialize 函式中所設置，設置方法如下:
    
    from zipline.api import set_slippage
    from zipline.finance import slippage
    
    def initialize(context, data):
        set_slippage(slippage.上述其中一種)
    
如果 initialize(context, data): 裡面沒有任何 set_slippage()，系統預設使用 FixedBasisPointsSlippage(basis_points = 5.0, volume_limit = 0.1)。如果希望完全不考慮交易量跟滑價，建議用 set_slippage(slippage.FixedSlippage(spread = 0.0))。

## 設定環境

In [34]:
import pandas as pd 
import numpy as np
import tejapi
import os

apikey = 'your key'
os.environ['TEJAPI_BASE'] = 'https://api.tej.com.tw'
os.environ['TEJAPI_KEY'] = apikey
os.environ['mdate'] = "20221201 20221231"
os.environ['ticker'] = "IR0001 1216 5844"

tejapi.ApiConfig.api_key = apikey

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-08 02:54:22.028390] INFO: zipline.data.bundles.core: Ingesting tquant.


In [35]:
from zipline.api import *
from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
from zipline.utils.run_algo import  (get_transaction_detail,get_data_for_alphalens,get_record_vars)
from zipline.sources.TEJ_Api_Data import get_Treasury_Return

## FixedSlippage 示範

In [36]:
start_dt = pd.Timestamp('2022-12-01', tz='UTC')
end_dt = pd.Timestamp('2022-12-31', tz='UTC')

def initialize(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]  
    # 設定滑架模型來進行模擬                
    # set_slippage()只接收一個spread參數
    set_slippage(slippage.FixedSlippage(spread = 0.2))
    # set_commision()可接收股票與期貨兩個參數
    # 這裡在接收commission.PerDollar()回傳結果後輸入參數
    set_commission(commission.PerDollar(cost = commission_cost))
    # 設定benchmark
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        for asset in context.asset:
            order(asset, 5000)

    if context.i == 7:
        for asset in context.asset:
            order(asset, -2000)        
    
    record(close=data.current(context.asset, 'close'))
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')


In [37]:
# 評估結果
performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            bundle='tquant')

positions, transactions, orders = get_transaction_detail(performance)

### 情況 1: 買入時計算滑價

12/1 時下單買五張統一 (1216) 股票，12/2 日成交。收盤價是 65.0，但因為我們設定spread = 0.2，所以成交價是 65 + 0.2 / 1 = 65.1 (看圖二)，手續費 (看圖一'commission') 是 65.1 * 5000 * 0.001425 = 463.8375 ( commission_cost是預先設定好的，這次用PerDollar )。

In [38]:
# 圖一
performance['orders'][1]

[{'id': '69a852e55c5e4b2c8d1fb438baa130e4',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 5000,
  'filled': 5000,
  'commission': 463.8374999999999,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [39]:
# 圖二
transactions

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-12-02 00:00:00+08:00,Equity(0 [1216]),1216,5000,2022-12-02 13:30:00+08:00,65.1,69a852e55c5e4b2c8d1fb438baa130e4,None
2022-12-13 00:00:00+08:00,Equity(0 [1216]),1216,-2000,2022-12-13 13:30:00+08:00,65.3,1885a7140d0342d98f3672f858650e69,None


### 情況 2: 賣出時計算滑價

在 12/12 賣出兩張，12/13 成交。12/13 收盤價65.4，因為是賣出，所以成交價是 65.4 扣掉 spread / 2 = 65.3，手續費計算方法同上。 

In [40]:
performance['orders'][8]

[{'id': '1885a7140d0342d98f3672f858650e69',
  'dt': Timestamp('2022-12-13 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-12 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -2000,
  'filled': -2000,
  'commission': 186.10500000000005,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

##  VolumeShareSlippage 示範


In [41]:
def initialize(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage. VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))
    set_commission(commission.PerDollar(cost = commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        for asset in context.asset:
            order(asset, 1500000)       
    
    if context.i == 10:
        for asset in context.asset:
            order(asset, -200000)    
    
    record(close=data.current(context.asset, 'close'))
    record(volume=data.current(context.asset, 'volume'))
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e8
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')


In [42]:
performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            bundle='tquant')

positions, transactions, orders = get_transaction_detail(performance)

### 情況 1: 買入時計算滑價

在 12/1 日下單一萬五千張統一，但觀察上面資料發現，這段期間每日成交量大約只有數千到一萬多張，而且我們設定 volume_limit = 0.025，所以zipline會把這單拆成數天慢慢消化，每天成交量不超過當天總成交量的 2.5%。

In [43]:
performance['orders'][0]

[{'id': '6fbc1851eba045e38a6b66cfef0558bf',
  'dt': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1500000,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [44]:
performance['volume'][0:6]

2022-12-01 13:30:00+08:00    Equity(0 [1216])    18963000
Name: volume, dty...
2022-12-02 13:30:00+08:00    Equity(0 [1216])    15184000
Name: volume, dty...
2022-12-05 13:30:00+08:00    Equity(0 [1216])    9704000
Name: volume, dtyp...
2022-12-06 13:30:00+08:00    Equity(0 [1216])    13171000
Name: volume, dty...
2022-12-07 13:30:00+08:00    Equity(0 [1216])    13674000
Name: volume, dty...
2022-12-08 13:30:00+08:00    Equity(0 [1216])    9149000
Name: volume, dtyp...
Name: volume, dtype: object

下單後第二天 12/2 的總交易量是 18963000 股，所以 VolumeShareSlippage 把成交量限制在 2.5%，也就是 379600。

成交價計算方法是：原價 * ( 1 + price_impact * volume_share * volume_share ) = 65 * ( 1 + 0.1 * 0.025^2 ) ~= 65.004063

price_impact 是預先設定好的 0.1，因為是買入，所以用加的。volume_share 就是這筆交易佔總交易量的比例 379600 / 18963000 = 0.025。

In [45]:
transactions[0:5]

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-12-02 00:00:00+08:00,Equity(0 [1216]),1216,379600,2022-12-02 13:30:00+08:00,65.004063,6fbc1851eba045e38a6b66cfef0558bf,None
2022-12-05 00:00:00+08:00,Equity(0 [1216]),1216,242600,2022-12-05 13:30:00+08:00,65.404088,6fbc1851eba045e38a6b66cfef0558bf,None
2022-12-06 00:00:00+08:00,Equity(0 [1216]),1216,329275,2022-12-06 13:30:00+08:00,64.604037,6fbc1851eba045e38a6b66cfef0558bf,None
2022-12-07 00:00:00+08:00,Equity(0 [1216]),1216,341850,2022-12-07 13:30:00+08:00,65.204075,6fbc1851eba045e38a6b66cfef0558bf,None
2022-12-08 00:00:00+08:00,Equity(0 [1216]),1216,206675,2022-12-08 13:30:00+08:00,65.003317,6fbc1851eba045e38a6b66cfef0558bf,None


In [46]:
performance['orders'][1]

[{'id': '6fbc1851eba045e38a6b66cfef0558bf',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1500000,
  'filled': 379600,
  'commission': 35162.647528125,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.OPEN: 0>}]

如果一個訂單被拆成多天，'filled' 就是累積成交量，例如圖一，'filled' = 1293325 就是從 12/2 到 12/7 成交的 379600 + 242600 + 329275 + 341850 = 1293325。到了 12/8 已經買到一百五十萬股 (圖二)，最後的 'status' 就會從<ORDER_STATUS.OPEN: 0> 變成 <ORDER_STATUS.FILLED: 1>。

In [47]:
performance['orders'][4]

[{'id': '6fbc1851eba045e38a6b66cfef0558bf',
  'dt': Timestamp('2022-12-07 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1500000,
  'filled': 1293325,
  'commission': 119849.74076566406,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [48]:
# 圖二
performance['orders'][5]

[{'id': '6fbc1851eba045e38a6b66cfef0558bf',
  'dt': Timestamp('2022-12-08 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1500000,
  'filled': 1500000,
  'commission': 138993.9895287313,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

### 情況 2: 賣出時計算滑價

在 12/15 下單賣出 20 萬股，因為在 12/16 總交易量是 10721000，volume_share = 200000 / 10721000 ~= 0.0187 小於 0.025，所以 12/16 一天就能賣掉。成交價 (看圖四transactions.price) 是 65.3 * ( 1 - 0.1 * volume_share^2 ) = 65.297728 (賣出的話是減)。

In [49]:
performance['volume'][10:12]

2022-12-15 13:30:00+08:00    Equity(0 [1216])    7422000
Name: volume, dtyp...
2022-12-16 13:30:00+08:00    Equity(0 [1216])    10721000
Name: volume, dty...
Name: volume, dtype: object

In [50]:
performance['orders'][11]

[{'id': '0cfe2f06b6ab400c80fb960046241991',
  'dt': Timestamp('2022-12-16 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-15 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -200000,
  'filled': -200000,
  'commission': 18609.852339455556,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [51]:
transactions[-1:]

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-12-16 00:00:00+08:00,Equity(0 [1216]),1216,-200000,2022-12-16 13:30:00+08:00,65.297728,0cfe2f06b6ab400c80fb960046241991,None
